In [8]:
pip install -U sentence-transformers google-generativeai numpy scikit-learn faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 76.8 MB/s eta 0:00:00


In [3]:
pip install huggingface_hub

In [ ]:
from huggingface_hub import login
HF_TOKEN = 
login(token=HF_TOKEN)

In [ ]:
import numpy as np
import google.generativeai as genai
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from huggingface_hub import login

HF_TOKEN = 
login(token=HF_TOKEN)



GEMINI_API_KEY = 
genai.configure(api_key=GEMINI_API_KEY)

llm_engine = genai.GenerativeModel('gemini-2.5-flash')
embed_engine = SentenceTransformer('BAAI/bge-m3')

def process_document(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        raw_text = file.read()

    # تقطيع النص لحتت (Chunks) عشان نعرف ندور فيها بدقة
    # بنشيل أي مسافات فاضية أو سطور ملهاش لزمة
    text_chunks = [segment.strip() for segment in raw_text.split('\n') if len(segment.strip()) > 20]

    # تحويل الكلام لأرقام (Vectorization)
    # الموديل هنا بيفهم "معنى" كل حتة ويخزنها في شكل رياضي
    print("Wait a second... Encoding the knowledge base...")
    encoded_vectors = embed_engine.encode(text_chunks)

    return text_chunks, encoded_vectors

def generate_rag_response(user_query, chunks, vectors, top_results=2):
    # تحويل سؤال المستخدم لـ Vector عشان نقارنه باللي عندنا
    query_vector = embed_engine.encode([user_query])

    # حساب درجة الشبه بين السؤال وبين كل الحتت اللي في الملف
    similarity_scores = cosine_similarity(query_vector, vectors)[0]

    # ترتيب النتايج وناخد أكتر حتتين فيهم الإجابة
    top_indices = np.argsort(similarity_scores)[-top_results:][::-1]
    retrieved_context = "\n".join([chunks[i] for i in top_indices])


    system_prompt = f"""
    You are a helpful assistant. Use the following context to answer the question.
    If the answer is not in the context, just say you don't know.

    Context:
    {retrieved_context}

    User Question:
    {user_query}

    Answer:
    """

    # جيميناي بيصيغ الإجابة النهائية بناءً على المعلومات اللي بعتناها له
    ai_response = llm_engine.generate_content(system_prompt)
    return ai_response.text


source_file = 'arabic.txt'
# بنبني قاعدة البيانات الصغيرة بتاعتنا
chunks_list, vectors_list = process_document(source_file)

user_input = "احكيلي عن نظام الميه في القلعة كان شغال إزاي؟"
final_result = generate_rag_response(user_input, chunks_list, vectors_list)

print(f"\n[Question]: {user_input}")
print(f"\n[Gemini's Answer]:\n{final_result}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Wait a second... Encoding the knowledge base...

[Question]: احكيلي عن نظام الميه في القلعة كان شغال إزاي؟

[Gemini's Answer]:
تميزت القلعة بنظام هيدروليكي سابق لعصره، يعتمد على شبكة معقدة من القنوات الفخارية لتجميع مياه الأمطار وتخزينها في صهاريج بازلتية ضخمة محفورة في باطن الجبل، مما كفل للحامية العسكرية الاكتفاء الذاتي من المياه خلال فترات الحصار الخانقة.


In [ ]:
import numpy as np
import faiss
import os
import google.generativeai as genai
from sentence_transformers import SentenceTransformer
from huggingface_hub import login


# =========================
# 1️⃣ Authentication
# =========================



# IMPORTANT: Replace "YOUR_GEMINI_KEY" with your actual Gemini API key.
# You can obtain a key from Google AI Studio.
GEMINI_API_KEY = 
genai.configure(api_key=GEMINI_API_KEY)


# =========================
# 2️⃣ Load Models
# =========================

llm_engine = genai.GenerativeModel('gemini-2.5-flash')
embed_engine = SentenceTransformer('BAAI/bge-m3')


# =========================
# 3️⃣ Build or Load FAISS DB
# =========================

def build_or_load_index(file_path):

    # لو في index محفوظ قبل كده
    if os.path.exists("faiss_index.index") and os.path.exists("chunks.npy"):
        print("📂 Loading existing FAISS index...")
        index = faiss.read_index("faiss_index.index")
        chunks = np.load("chunks.npy", allow_pickle=True)
        return chunks, index

    print("📦 Building new FAISS index...")

    with open(file_path, 'r', encoding='utf-8') as file:
        raw_text = file.read()

    text_chunks = [
        segment.strip()
        for segment in raw_text.split('\n')
        if len(segment.strip()) > 20
    ]

    print(f"✅ Total Chunks: {len(text_chunks)}")

    embeddings = embed_engine.encode(text_chunks)
    embeddings = np.array(embeddings).astype("float32")

    dimension = embeddings.shape[1]

    # إنشاء Index
    index = faiss.IndexFlatL2(dimension)

    index.add(embeddings)

    # نحفظه على الديسك
    faiss.write_index(index, "faiss_index.index")
    np.save("chunks.npy", text_chunks)

    print("✅ FAISS index saved to disk.")

    return text_chunks, index


# =========================
# 4️⃣ Search + Generate
# =========================

def generate_rag_response(user_query, chunks, index, top_k=2):

    print(f"\n🧠 Question: {user_query}")

    query_vector = embed_engine.encode([user_query])
    query_vector = np.array(query_vector).astype("float32")

    distances, indices = index.search(query_vector, top_k)

    print("\n🔎 Retrieved Chunks:\n")

    retrieved_chunks = []

    for i, idx in enumerate(indices[0]):
        print(f"Rank {i+1}")
        print(f"Chunk Index: {idx}")
        print(f"Distance Score: {distances[0][i]:.4f}")
        print(f"Content:\n{chunks[idx]}")
        print("-" * 60)

        retrieved_chunks.append(chunks[idx])

    retrieved_context = "\n".join(retrieved_chunks)

    prompt = f"""
    You are a helpful assistant.
    Answer ONLY using the provided context.
    If not found in context, say you don't know.

    Context:
    {retrieved_context}

    Question:
    {user_query}

    Answer:
    """

    response = llm_engine.generate_content(prompt)
    return response.text


# =========================
# 5️⃣ Execution
# =========================

source_file = "arabic.txt"

chunks_list, faiss_index = build_or_load_index(source_file)

user_input = "احكيلي عن نظام الميه في القلعة كان شغال إزاي؟"

answer = generate_rag_response(
    user_input,
    chunks_list,
    faiss_index,
    top_k=2
)

print("\n======================")
print("FINAL ANSWER")
print("======================\n")
print(answer)

Loading weights:   0%|          | 0/391 [00:00<?, ?it/s]

📦 Building new FAISS index...
✅ Total Chunks: 6
✅ FAISS index saved to disk.

🧠 Question: احكيلي عن نظام الميه في القلعة كان شغال إزاي؟

🔎 Retrieved Chunks:

Rank 1
Chunk Index: 3
Distance Score: 1.0717
Content:
ومع دخول المنطقة تحت مظلة الحكم العثماني، شهدت القلعة انعطافات تاريخية ووظيفية حادة؛ فقد تحولت من مقر للسيادة إلى ثكنة عسكرية، ثم تلا ذلك تحويل "البرج الجنوبي"—الذي كان يُعرف سابقاً ببرج الرياح لدقة تصميمه في التهوية—إلى معتقل سياسي شديد الحراسة خصص للمعارضين من الأعيان والولاة المتمردين. أما الساحات الشمالية الفسيحة، فقد أُعيد توظيفها لتصبح "شوماً" (مخازن) استراتيجية لتخزين الحبوب والملح والبارود، وهو ما جعلها هدفاً متكرراً للنيران. ولقد عانت القلعة من نكبات متتالية، أبرزها "زلزال القرن السابع عشر" والحريق الهائل الذي تلاه، مما أدى إلى تصدع الأسوار الخارجية وانهيار الأسقف الخشبية المزخرفة، لتدخل القلعة بعدها في نفق مظلم من الإهمال والنسيان استمر لعقود. وخلال تلك الفترة من العزلة، نسج المخيال الشعبي أساطير لا حصر لها حول ما يدور خلف تلك الأسوار الصامتة؛ فتناقلت الأجيال حكايات ع